In [1]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
ensure_pyterrier_is_loaded()

import pyterrier as pt
import pandas as pd

Start PyTerrier with version=5.7, helper_version=0.0.7, no_download=True


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
prompt = '4'

for d in ['tot-test', 'tot-train', 'tot-dev']:
    queries = pt.io.read_topics(f'../../{d}/queries.xml', 'trecxml')
    run = pd.read_json(f'../{d}-rerank-{prompt}-with-scores.jsonl.gz', lines=True, dtype={'qid': str, 'docno': str})
    run['qid'] = run['qid'].apply(lambda i: str(i).split('_')[0])
    run = pt.Transformer.from_df(run)
    run = run(queries)
    !mkdir -p {d}
    
    persist_and_normalize_run(run, 'webis-t5-01', f'{d}/run.txt')

### Final test to verify that the scores are identical to the hyperparameter tuning.

In [3]:
def report_effectiveness(dataset):
    queries = pt.io.read_topics(f'../../{dataset}/queries.xml', 'trecxml')
    run = pt.Transformer.from_df(pt.io.read_results(f'{dataset}/run.txt'))

    ret = pt.Experiment([run], queries, pt.io.read_qrels(f'../../{dataset}/qrels.txt'), eval_metrics=["ndcg_cut_10", 'recip_rank', 'P_10', 'recall_100', 'recall_1000', 'recall_5000'])
    
    return {i: ret.iloc[0][i] for i in ['ndcg_cut_10', 'recip_rank', 'P_10', 'recall_100', 'recall_1000', 'recall_5000']}

def report_pipeline():
    ret = {}
    for dataset in ['tot-train', 'tot-dev']:
        for k,v in report_effectiveness(dataset).items():
            ret[dataset.split('-')[1] + '_' + k] = v
    return pd.DataFrame([ret])

report_pipeline()

,train_ndcg_cut_10,train_recip_rank,train_P_10,train_recall_100,train_recall_1000,train_recall_5000,dev_ndcg_cut_10,dev_recip_rank,dev_P_10,dev_recall_100,dev_recall_1000,dev_recall_5000
0,0.106904,0.09833,0.016667,0.426667,0.626667,0.626667,0.112698,0.102139,0.017333,0.373333,0.566667,0.566667
